<a href="https://colab.research.google.com/github/ll1195831146/Infor7374-AI/blob/master/Assignment2/CNN_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load and Preprocess Data**

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive

In [2]:
import os
os.chdir(os.path.join(os.getcwd(),'drive'))
print(os.getcwd())
print(os.listdir(os.getcwd()))

/content/drive
['.Trash', 'Video_Games_Sales.csv', 'val_data', 'Getting started', 'Track-guidelines.pdf', 'AED_Lab1_Part1.mp4', 'AED_Lab1_Part2Final.mp4', 'games.zip', 'VideoGames-Version2 2.zip', 'VideoGames-Version4.zip', 'VideoGames-Version4-2.zip', 'NBA_stats_f (Recovered).ods', 'RL-NikBrown.odt', 'Resume0102.odt', '6210 Database Design.odt', 'Colab Notebooks', 'Imagenet32_train', 'y_val_1.npy', 'out_data_val', 'out_data_train', 'out_data_train_1', 'out_data_val.zip', 'out_data_val_1.zip', 'out_data_train_1.zip', 'out_data_train.zip', 'data', 'drive', 'Assignment2-E1.ipynb', 'resized-tiny-imagenet-200']


In [0]:
def get_annotations_map():
	valAnnotationsPath = './resized-tiny-imagenet-200/val/val_annotations.txt'
	valAnnotationsFile = open(valAnnotationsPath, 'r')
	valAnnotationsContents = valAnnotationsFile.read()
	valAnnotations = {}

	for line in valAnnotationsContents.splitlines():
		pieces = line.strip().split()
		valAnnotations[pieces[0]] = pieces[1]

	return valAnnotations

In [0]:
import numpy as np
from PIL import Image

def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,32,32],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        if sChild == '.DS_Store':
            continue
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
#         print(j)
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            if c == '.DS_Store':
                continue
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,32,32],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if sChild == '.DS_Store':
            continue
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass

    X_train = X_train.transpose(0,2,3,1)
    X_test = X_test.transpose(0,2,3,1)
    print('finished loading test images')

    return X_train,y_train,X_test,y_test

In [0]:
path='./resized-tiny-imagenet-200'
x_train,y_train,x_test,y_test=load_images(path,5)

In [6]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
import os
from keras import metrics

Using TensorFlow backend.


In [7]:
x_train.shape

(2500, 32, 32, 3)

In [0]:
num_classes = 5
data_augmentation = True
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

**1st Model**

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)
    
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [21]:
model.compile(loss='categorical_crossentropy',
optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
metrics=['accuracy',metrics.categorical_accuracy])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_16 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_17 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 15, 15, 64)        18496     
__________

In [22]:
model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=100),
    epochs=5,
    steps_per_epoch=200,
    validation_data=(x_test, y_test),
    verbose=1,
    workers=16)

Epoch 1/5
200/200 [==============================] - 16s 82ms/step - loss: 1.5967 - acc: 0.2756 - categorical_accuracy: 0.2756 - val_loss: 1.5697 - val_acc: 0.3440 - val_categorical_accuracy: 0.3440
Epoch 2/5
200/200 [==============================] - 15s 74ms/step - loss: 1.4923 - acc: 0.3524 - categorical_accuracy: 0.3524 - val_loss: 1.3982 - val_acc: 0.4320 - val_categorical_accuracy: 0.4320
Epoch 3/5
200/200 [==============================] - 15s 75ms/step - loss: 1.4032 - acc: 0.3990 - categorical_accuracy: 0.3990 - val_loss: 1.3489 - val_acc: 0.4200 - val_categorical_accuracy: 0.4200
Epoch 4/5
200/200 [==============================] - 15s 75ms/step - loss: 1.3688 - acc: 0.4200 - categorical_accuracy: 0.4200 - val_loss: 1.3181 - val_acc: 0.4720 - val_categorical_accuracy: 0.4720
Epoch 5/5
200/200 [==============================] - 15s 75ms/step - loss: 1.3292 - acc: 0.4479 - categorical_accuracy: 0.4479 - val_loss: 1.2706 - val_acc: 0.5000 - val_categorical_accuracy: 0.5000


In [23]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

250/250 [==============================] - 0s 409us/step
Test loss: 1.2705574026107789
Test accuracy: 0.5


**2nd Model**

In [16]:
model = Sequential()

# layer 1
model.add(Conv2D(128, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

# layer 2
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# layer 3
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))

# layer 4
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# layer 5
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
model.compile(loss='categorical_crossentropy',
optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
metrics=['accuracy',metrics.categorical_accuracy])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 128)       147584    
_________________________________________________________________
activation_9 (Activation)    (None, 30, 30, 128)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)       147584    
__________

In [18]:
model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=100),
    epochs=5,
    steps_per_epoch=200,
    validation_data=(x_test, y_test),
    verbose=1,
    workers=16)

Epoch 1/5
200/200 [==============================] - 19s 95ms/step - loss: 1.6106 - acc: 0.2083 - categorical_accuracy: 0.2083 - val_loss: 1.6092 - val_acc: 0.2000 - val_categorical_accuracy: 0.2000
Epoch 2/5
200/200 [==============================] - 17s 84ms/step - loss: 1.6094 - acc: 0.2066 - categorical_accuracy: 0.2066 - val_loss: 1.6092 - val_acc: 0.2000 - val_categorical_accuracy: 0.2000
Epoch 3/5
200/200 [==============================] - 17s 85ms/step - loss: 1.6070 - acc: 0.2215 - categorical_accuracy: 0.2215 - val_loss: 1.6084 - val_acc: 0.2000 - val_categorical_accuracy: 0.2000
Epoch 4/5
200/200 [==============================] - 17s 85ms/step - loss: 1.6042 - acc: 0.2464 - categorical_accuracy: 0.2464 - val_loss: 1.6068 - val_acc: 0.2000 - val_categorical_accuracy: 0.2000
Epoch 5/5
200/200 [==============================] - 17s 85ms/step - loss: 1.5993 - acc: 0.2626 - categorical_accuracy: 0.2626 - val_loss: 1.6024 - val_acc: 0.2520 - val_categorical_accuracy: 0.2520


In [19]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

250/250 [==============================] - 0s 847us/step
Test loss: 1.6023705635070802
Test accuracy: 0.25200000059604644


**3rd Model**

In [30]:
from keras.layers.convolutional import Convolution2D

model = Sequential()
#conv-spatial batch norm - relu #1 
model.add(ZeroPadding2D((2,2),input_shape=x_train.shape[1:]))
model.add(Convolution2D(64,5,5,subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu')) 

#conv-spatial batch norm - relu #2
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64,3,3,subsample=(1,1)))
model.add(BatchNormalization())
model.add(Activation('relu')) 

#conv-spatial batch norm - relu #3
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128,3,3,subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu')) 
model.add(Dropout(0.25)) 

#conv-spatial batch norm - relu #4
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128,3,3,subsample=(1,1)))
model.add(BatchNormalization())
model.add(Activation('relu')) 

#conv-spatial batch norm - relu #5
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256,3,3,subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu')) 

#conv-spatial batch norm - relu #6
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256,3,3,subsample=(1,1)))
model.add(BatchNormalization())
model.add(Activation('relu')) 
model.add(Dropout(0.25))

#conv-spatial batch norm - relu #7
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,3,3,subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu')) 

#conv-spatial batch norm - relu #8
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,3,3,subsample=(1,1)))
model.add(BatchNormalization())
model.add(Activation('relu')) 


#conv-spatial batch norm - relu #9
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(1024,3,3,subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25)) 

#Affine-spatial batch norm -relu #10 
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(num_classes))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), strides=(2, 2))`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1))`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), strides=(1, 1))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), strides=(1, 1))`
/usr

In [31]:
model.compile(loss='categorical_crossentropy',
optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
metrics=['accuracy',metrics.categorical_accuracy])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_6 (ZeroPaddin (None, 36, 36, 3)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 16, 16, 64)        4864      
_________________________________________________________________
batch_normalization_13 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
activation_22 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_14 (Batc (None, 16, 16, 64)        256       
__________

In [32]:
model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=100),
    epochs=5,
    steps_per_epoch=200,
    validation_data=(x_test, y_test),
    verbose=1,
    workers=16)

Epoch 1/5
200/200 [==============================] - 25s 125ms/step - loss: 1.5702 - acc: 0.3855 - categorical_accuracy: 0.3855 - val_loss: 1.2090 - val_acc: 0.4680 - val_categorical_accuracy: 0.4680
Epoch 2/5
200/200 [==============================] - 20s 101ms/step - loss: 1.1842 - acc: 0.5282 - categorical_accuracy: 0.5282 - val_loss: 1.2210 - val_acc: 0.5680 - val_categorical_accuracy: 0.5680
Epoch 3/5
200/200 [==============================] - 20s 101ms/step - loss: 0.9852 - acc: 0.6181 - categorical_accuracy: 0.6181 - val_loss: 0.9973 - val_acc: 0.6560 - val_categorical_accuracy: 0.6560
Epoch 4/5
200/200 [==============================] - 20s 101ms/step - loss: 0.8622 - acc: 0.6687 - categorical_accuracy: 0.6687 - val_loss: 0.8775 - val_acc: 0.6360 - val_categorical_accuracy: 0.6360
Epoch 5/5
200/200 [==============================] - 20s 101ms/step - loss: 0.7786 - acc: 0.6996 - categorical_accuracy: 0.6996 - val_loss: 1.4796 - val_acc: 0.5800 - val_categorical_accuracy: 0.5800


In [33]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

250/250 [==============================] - 0s 2ms/step
Test loss: 1.4796340646743775
Test accuracy: 0.5800000023841858


**4th Model(Best one)**

In [9]:
model = Sequential()

# """Block 1"""
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same', 
					input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
model.add(Activation('relu'))

# """Block 2"""
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

# """Block 3"""
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

# """Block 4"""
model.add(Conv2D(256, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

# """Block 5"""
model.add(Flatten())
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))

# """Block Test"""
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))

# """Output Layer"""
model.add(Dense(num_classes))

# """Loss Layer"""
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
model.compile(loss='categorical_crossentropy',
optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
metrics=['accuracy',metrics.categorical_accuracy])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
__________

In [14]:
model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=100),
    epochs=5,
    steps_per_epoch=200,
    validation_data=(x_test, y_test),
    verbose=1,
    workers=16)

Epoch 1/5
200/200 [==============================] - 25s 124ms/step - loss: 0.0078 - acc: 0.9994 - categorical_accuracy: 0.9994 - val_loss: 0.8517 - val_acc: 0.7840 - val_categorical_accuracy: 0.7840
Epoch 2/5
200/200 [==============================] - 24s 119ms/step - loss: 0.0066 - acc: 0.9998 - categorical_accuracy: 0.9998 - val_loss: 0.8523 - val_acc: 0.7760 - val_categorical_accuracy: 0.7760
Epoch 3/5
200/200 [==============================] - 24s 120ms/step - loss: 0.0066 - acc: 0.9994 - categorical_accuracy: 0.9994 - val_loss: 0.9091 - val_acc: 0.7840 - val_categorical_accuracy: 0.7840
Epoch 4/5
200/200 [==============================] - 24s 119ms/step - loss: 0.0061 - acc: 0.9996 - categorical_accuracy: 0.9996 - val_loss: 0.8888 - val_acc: 0.7800 - val_categorical_accuracy: 0.7800
Epoch 5/5
200/200 [==============================] - 24s 119ms/step - loss: 0.0059 - acc: 0.9997 - categorical_accuracy: 0.9997 - val_loss: 0.8982 - val_acc: 0.7840 - val_categorical_accuracy: 0.7840


In [15]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

250/250 [==============================] - 0s 2ms/step
Test loss: 0.8982262268066407
Test accuracy: 0.7840000019073486


**5th Model**

In [0]:
model = Sequential()


model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same', 
					input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
model.add(Activation('relu'))


model.add(Conv2D(256, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))


model.add(Conv2D(512, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))


model.add(Conv2D(1024, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(1024, (3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))


model.add(Flatten())
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [36]:
model.compile(loss='categorical_crossentropy',
optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
metrics=['accuracy',metrics.categorical_accuracy])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_32 (Batc (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 32, 32, 128)       147584    
_________________________________________________________________
batch_normalization_33 (Batc (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
activation_38 (Activation)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 16, 16, 256)       295168    
__________

In [37]:
model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=100),
    epochs=5,
    steps_per_epoch=200,
    validation_data=(x_test, y_test),
    verbose=1,
    workers=16)

Epoch 1/5
200/200 [==============================] - 56s 280ms/step - loss: 0.7002 - acc: 0.7322 - categorical_accuracy: 0.7322 - val_loss: 0.6966 - val_acc: 0.7320 - val_categorical_accuracy: 0.7320
Epoch 2/5
200/200 [==============================] - 48s 240ms/step - loss: 0.2505 - acc: 0.9206 - categorical_accuracy: 0.9206 - val_loss: 0.7769 - val_acc: 0.7360 - val_categorical_accuracy: 0.7360
Epoch 3/5
200/200 [==============================] - 48s 240ms/step - loss: 0.1153 - acc: 0.9719 - categorical_accuracy: 0.9719 - val_loss: 0.7065 - val_acc: 0.7760 - val_categorical_accuracy: 0.7760
Epoch 4/5
200/200 [==============================] - 48s 240ms/step - loss: 0.0638 - acc: 0.9882 - categorical_accuracy: 0.9882 - val_loss: 0.7246 - val_acc: 0.7600 - val_categorical_accuracy: 0.7600
Epoch 5/5
200/200 [==============================] - 48s 240ms/step - loss: 0.0408 - acc: 0.9933 - categorical_accuracy: 0.9933 - val_loss: 0.7685 - val_acc: 0.7680 - val_categorical_accuracy: 0.7680


In [38]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

250/250 [==============================] - 1s 5ms/step
Test loss: 0.7684581179618836
Test accuracy: 0.7680000014305115
